<a href="https://colab.research.google.com/github/mariajoseag/Analitica-de-Negocios/blob/main/Reto_1_Car_Crash_Naive_Bayes_Maria_Jos%C3%A9_Agudelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reto 1- Modelo Naive Bayes**

0. Se cargan las librerias de trabajo

In [ ]:
import numpy as np #Esta libreria tiene una serie de funciones numéricas para la transformación de los datos y el análisis
import pandas as pd #Esta libreria permite conectar el sistema con las bases de datos que tengo disponibles en la carpeta content

1. Se carga la base de datos

In [ ]:
nxl='/content/sample_data/1. BD2_CarCrash_Int.xlsx'
XDB=pd.read_excel(nxl,sheet_name=0) #Con este código le digo al sistema cual es la hoja de la base de datos que quiero, siempre teniendo en cuenta que lo que sería la hoja 1 en excel para colab se cuenta como la hoja 0

In [ ]:
#Filtramos los datos solo para el condado de San Francisco creando un nuevo DataFrame
SD=XDB[XDB['County']=='SAN FRANCISCO']

In [ ]:
SD.dropna() #Eliminamos los datos faltantes

,ID,County,City,Weekday,Severity,ViolCat,ClearWeather,Month,CrashType,Highway,Daylight
907,908,SAN FRANCISCO,SAN FRANCISCO,7,1,1,1,3,A,1,1
1580,1581,SAN FRANCISCO,SAN FRANCISCO,2,0,9,1,2,A,0,0
1581,1582,SAN FRANCISCO,SAN FRANCISCO,7,0,3,1,1,A,0,0
1586,1587,SAN FRANCISCO,SAN FRANCISCO,7,0,3,1,3,A,0,0
1587,1588,SAN FRANCISCO,SAN FRANCISCO,3,0,3,1,3,A,0,0
...,...,...,...,...,...,...,...,...,...,...,...
112598,112599,SAN FRANCISCO,SAN FRANCISCO,5,0,11,1,8,G,0,0
112599,112600,SAN FRANCISCO,SAN FRANCISCO,5,0,11,1,3,G,0,1
112600,112601,SAN FRANCISCO,SAN FRANCISCO,6,0,10,0,4,G,0,0
112601,112602,SAN FRANCISCO,SAN FRANCISCO,4,0,10,1,4,G,0,0


In [ ]:
#Convertir los datos de la variable Crash Type a valores numéricos, en este caso utilizando la función replace y reemplazando las letras en orden alfabético por los numeros del 0 al 6
SD['CrashType']=SD['CrashType'].replace({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6}).infer_objects(copy=False)

In [ ]:
#Se cargan las variables a tener en cuenta
SD=SD[['Weekday','ViolCat','ClearWeather','Month','CrashType','Highway','Daylight']]

In [ ]:
#Separamos las variables por entrada y salida
XD=SD[['Weekday','ViolCat','ClearWeather','Month','CrashType','Daylight']] #Aquí estan las variables de entrada
yd=SD[['Highway']] #Aquí estan la variable de referencia

2. Se implementa el modelo de Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB #En este caso estamos realizando un préstamo interbibliotecario para implementar la función Gaussian

mnb=GaussianNB()
mnb.fit(XD,yd.values.ravel())

#Obtenemos alguna información
ct=mnb.class_count_
print("¿Cómo estan separados los datos?\n", ct) #¿Cómo estan separados los datos?
print("El total de los datos es:\n",len(XD)) #¿Cuál es el total de los datos?
print("Los datos porcentualmente están divididos:\n",ct/len(XD)) #¿Cómo están los datos divididos porcentualmente?

#Se obtiene información por categoria
u=mnb.theta_
print("La media de accidentes por categoria es:\n",u)
std=mnb.var_ #Esto es varianza (Varianza es el valor elevado a la 2)
std=np.sqrt(std) #Para obtener la desviación estandár que se obtiene es sacando raiz cuadrada
print("La desviación estandár de accidentes por categoria es:\n",std)

#Se identifican los límites superiores e inferiores de las categorías
u=mnb.theta_
print("El límite superior de los valores:\n",u+std)
print("El límite inferior de los valores:\n",u-std)

#Para indicar la categoria a la que pertenece una persona con las características [5,1,1,3,4,0]
XS=pd.DataFrame([[5,1,1,3,4,0]],columns=XD.columns)
mnb.predict(XS)

¿Cómo estan separados los datos?
 [1580.  501.]
El total de los datos es:
 2081
Los datos porcentualmente están divididos:
 [0.75925036 0.24074964]
La media de accidentes por categoria es:
 [[3.88417722 7.45696203 0.86265823 4.35189873 3.21772152 0.66265823]
 [3.9261477  4.1497006  0.79840319 6.69461078 2.27744511 0.62075848]]
La desviación estandár de accidentes por categoria es:
 [[1.95062542 3.52136997 0.34420782 2.3433645  1.87371962 0.47280262]
 [2.01603723 2.16198902 0.40119266 3.44163641 1.05765448 0.48519831]]
El límite superior de los valores:
 [[ 5.83480263 10.978332    1.20686605  6.69526323  5.09144114  1.13546084]
 [ 5.94218494  6.31168962  1.19959585 10.13624719  3.33509959  1.10595679]]
El límite inferior de los valores:
 [[1.9335518  3.93559205 0.51845041 2.00853424 1.3440019  0.18985561]
 [1.91011047 1.98771158 0.39721054 3.25297437 1.21979063 0.13556017]]


array([0])

3. **Matriz de Confisión**
Se procede con la evaluacion o desempeño del modelo - Para esto se utilizara la matriz de confusión, para mirar como un modelo de estos funciona o no funciona

In [ ]:
ydp=mnb.predict(XD) #Se le dice al código hazme el pronóstico de los positivos y negativos

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(yd,ydp)
print(cm)

VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1] #Esta es la matríz de confisión

#Calcular la exactitud del modelo para evaluar el desempeño general del mismo
Ex=(VN+VP)/(VN+VP+FN+FP)
print("La exactitud del modelo es:",Ex)

#Calcular la Tasa de error del modelo, para evaluar cuánto se equivocó el mismo
TE=(FP+FN)/(VN+VP+FN+FP)
print("La tasa de error del modelo es:",TE)

#Calcular la Sensibilidad del modelo para medir el comportamiento del modelo identificando si un accidente automobilístico se provocó en una autopista
Sen=VP/(VP+FN)
print("La sensibilidad del modelo es:",Sen)

#Calcular la Especificidad del modelo para medir el comportamiento del modelo identificando si un accidente automovilístico no se provocó en una autopista
Esp=VN/(VN+FP)
print("La especificidad del modelo es:",Esp)

#Calcular la Precisión del modelo para evaluar como se comporta en general el modelo identificando los accidente automovílisticos provocados en una autopista
pre=VP/(VP+FP)
print("La precisión del modelo es:",pre)

#Calcular la Predicción negativa para evaluar como funciona el modelo reconociendo accidentes automovilísticos no provocados en una autopista
pren=VN/(VN+FN)
print("La predicción negativa del modelo es:",pren)

[[1365  215]
 [ 224  277]]
La exactitud del modelo es: 0.7890437289764536
La tasa de error del modelo es: 0.21095627102354636
La sensibilidad del modelo es: 0.5528942115768463
La especificidad del modelo es: 0.8639240506329114
La precisión del modelo es: 0.5630081300813008
La predicción negativa del modelo es: 0.8590308370044053


**Análisis de Resultados**

De acuerdo con los resultados arrojados por el modelo, se puede observar que el modelo alcanzó una exactitud, o desempeño general muy cercano al 80%, sobresaliendo el comportamiento del modelo frente a la especificidad del modelo para medir el comportamiento del mismo identificando si un accidente automovilístico no se provocó en una autopista, con un resultado de un poco más del 86%, y también una predicción negativa de más del 85%, indicando un buen funcionamiento del modelo reconociendo accidentes automovilísticos no provocados en una autopista.

Finalmente presentando una tasa de error de tan solo 21%, aunque también cabe aclarar que la precision del modelo es tan solo del 56%, lo que quiere decir que el modelo se comportó muy bien en la prediccion de los valores positivos, es decir los accidentes automovilisticos que si ocurrieron en carretera.
La sensibilidad fue del 55% mostrando un comportamiento positivo al identificar los accidentes que si ocurrienron en autopista.

El número total en la base de datos es de 112660, de los cuales 2081 pertenecen al condado de San Francisco. Las variables utilizadas para el desarrollo del caso fueron Weekday, ViolCat, ClearWeather, Month, CrashType, y Daylight, y la variable Highway fue tomada como variable de referencia para la predicción de si un accidente automovilístico se provocó en una autopista o no (Highway).

1580 de los accidentes presentados en San Francisco no fueron provocados en una autopista, mientras que 501 de estos accidentes si se provocaron en una autopista. Los accidentes que no fueron provocados en una autopista representan el 75,9% del total de datos de accidentes en San Francisco, mientras que los que si fueron provocados en una autopista representan el 24%.

De los 1580 accidentes automovilísticos que si fueron en autopista, 277 fueron calculados correctamente, mientras que 224 fueron falsos negativos; de los 1580 accidentes automovilísticos que no fueron en autopista 1365 fueron verdaderos negativos, es decir que se pronosticaron correctamente, y 215 fueron falsos positivos que no se pudieron indicar con exactitud.

Finalmente, analizando la descripción de cada una de las variables, tenemos que:

- Weekday es el día de la semana en que ocurrió cada accidente automovilístico. Los datos se presentan en números enteros, y se cuentan del 1 al 7, 1- Lunes, 2- Martes 3- Miércoles, 4- Jueves, 5- Viernes, 6- Sábado y 7- Domingo.

- ViolCat cuantifica el nivel de impacto del accidente. Los datos se presentan en números enteros, y se cuentan del 1 al 12. Indicando el grado de seriedad del impacto en cada accidente, siendo 1 el más bajo y 12 el nivel de impacto más alto.

- ClearWeather indica si las condiciones climáticas al momento del accidente eran despejadas o no. Los datos se presentan en los números enteros 0 y 1, 0 indicando no despejado, 1 indicando que si estaba despejado.

- Month es el mes en el que ocurrió el accidente. Los datos se presentan en números enteros del 1 al 12, siendo 1 Enero, 2 Febrero, 3 Marzo, 4 Abril, 5 Mayo, 6 Junio, 7 Julio, 8 Agosto, 9 Septiembre, 10 Octubre, 11 Noviembre y 12 Diciembre.

- CrashType indica el tipo de accidente, que se presentó en primer lugar por la base de datos en tipo categórico, utilizando letras de la A a la G en orden alfabético. Donde cada letra representa un tipo especifico de accidente.

- Daylight indica si el accidente ocurrió de día o de noche. Se presenta en los números enteros 0 y 1, 0 para noche y 1 para día.